# 구내식당 식수 인원 예측 AI 경진대회
 지금까지는 단순한 시계열 추세와 담당자의 직관적인 경험에 의존하여 구내식당 식수 인원을 예측하였습니다. 식수 인원의 예측은 잔반량을 줄이는데 큰 기여를 할 수 있기 때문에 매우 중요합니다. 머신러닝 기법을 적용하여 보다 정확도 높은 예측을 통해 잔반 발생량을 획기적으로 줄이고자 합니다.


In [ ]:
# 사용하는 패키지 로드
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor

## 데이터 전처리 ##
Dacon에서 제공하는 데이터는 train과 test를 위한 형태로 제공됩니다. 하지만 test데이터는 성능 평가를 위해 필요한 종속변수 (Y)의 값이 없습니다. 왜냐하면 예측된 데이터를 제출하여 Dacon에서 성능 평가를 하기 때문입니다. 따라서 우리는 train에 해당되는 파일을 나눠 학습과 추론, 그리고 성능 평가를 진행합니다.

In [10]:
data = pd.read_csv('./train.csv')

In [11]:
data.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계
0,2016-02-01,월,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0
1,2016-02-02,화,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0
2,2016-02-03,수,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0
3,2016-02-04,목,2601,104,220,355,0.0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",978.0,525.0
4,2016-02-05,금,2601,278,181,34,0.0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",925.0,330.0


데이터 셋의 속성 (컬럼)은 다음의 정보를 포함합니다.
* 일자, 요일
* 본사정원수
* 본사휴가자수, 본사출장자수
* 시간외근무명령서승인건수, 현본사소속재택근무자수
* 조식메뉴, 중식메뉴, 석식메뉴
* 중식계, 석식계

위의 다양한 특징 중에서 우리는 **어떠한 특징을 입력**으로 사용할 것인지 고민을 해야 합니다.

학습 데이터를 3:2 비율로 나눠서 학습과 성능 평가를 진행합니다.

In [27]:
train = data[0:3]
test = data[4:6]

display(train)
display(test)

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계
0,2016-02-01,월,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0
1,2016-02-02,화,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0
2,2016-02-03,수,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0


,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계
4,2016-02-05,금,2601,278,181,34,0.0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",925.0,330.0
5,2016-02-11,목,2601,383,143,417,0.0,팬케익/찐빵 우유/두유/주스 계란후라이 견과류죽/쌀밥 (쌀:국내산) 감자찌개 ...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 시래기국 훈제오리구이 도토리묵무침 쌈무/양...","참치회덮밥 (쌀,현미흑미:국내산) 맑은국 군만두 과일샐러드 락교 포기김치 (...",1045.0,550.0


학습과 성능 평가를 위해 어떠한 변수를 독립변수 (입력), 종속변수 (출력, 목표변수)로 할것인지를 지정합니다.  
여기서는 '본사정원수', '본사출장자수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수'를 사용합니다.

In [30]:
x_train = train[['본사정원수', '본사출장자수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수']]
y1_train = train['중식계']
y2_train = train['석식계']

x_test = test[['본사정원수', '본사출장자수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수']]
y1_test = test['중식계']
y2_test = test['석식계']

점심과 저녁 식수에 해당되는 인원을 예측하기 위한 머신러닝 모델 생성 (RandomForestRegressor)

In [24]:
model1 =                      (n_jobs=-1, random_state=42)
model2 =                      (n_jobs=-1, random_state=42)

머신러닝 모델 학습 (점심과 저녁)

In [34]:
model1.fit(x_train, y1_train)
model2.fit(x_train, y2_train)

RandomForestRegressor(n_jobs=-1, random_state=42)

In [40]:
pred1 = model1.predict(x_test)
pred2 = model2.predict(x_test)

display(pred1)
display(pred2)

array([991.8 , 945.24])

array([534.23, 418.54])

In [45]:
from sklearn.metrics import mean_absolute_error

MAE1 = mean_absolute_error(y1_test, pred1)**0.5
MAE2 = mean_absolute_error(y2_test, pred2)**0.5
print(MAE1)
print(MAE2)

9.125787637239865
12.955500762224515
